In [1]:
import pandas as pd

# f = pd.read_json(file_path, lines=True)
df_image_train = pd.read_json("data/train.jsonl", lines=True)
df_image_val = pd.read_json("data/dev.jsonl", lines=True)
df_image_test = pd.read_json("data/test.jsonl", lines=True)
df_image_train.head()

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
print(df_image_train.head())

      id            img  label  \
0  42953  img/42953.png      0   
1  23058  img/23058.png      0   
2  13894  img/13894.png      0   
3  37408  img/37408.png      0   
4  82403  img/82403.png      0   

                                                text  
0   its their character not their color that matters  
1  don't be afraid to love again everyone is not ...  
2                           putting bows on your pet  
3  i love everything and everybody! except for sq...  
4  everybody loves chocolate chip cookies, even h...  


In [10]:
from transformers import AutoTokenizer, AutoModel, ViTModel, ViTFeatureExtractor
import torchvision.models as models
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from PIL import Image
import pandas as pd
import os
import torchvision.transforms as T
from sklearn.metrics import precision_score, recall_score, roc_auc_score


# Transformers model for text
text_model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Torchvision model for images
image_model = ViTModel.from_pretrained("google/vit-base-patch16-224")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

def collate_fn(batch):
    images, texts, labels = zip(*batch)

    # Pad sequences
    input_ids = pad_sequence([t['input_ids'] for t in texts])
    attention_mask = pad_sequence([t['attention_mask'] for t in texts])

    # Convert to tensors
    input_ids = torch.as_tensor(input_ids, dtype=torch.long)
    attention_mask = torch.as_tensor(attention_mask, dtype=torch.long)
    labels = torch.as_tensor(labels, dtype=torch.long)

    # Transpose image tensor (batch_size should be first)
    images = torch.stack(images).transpose(0, 1)
    
    return images, {'input_ids': input_ids, 'attention_mask': attention_mask}, labels


class HatefulMemesDataset(Dataset):
    def __init__(self, json_file, img_dir, transforms=None):
        self.data = pd.read_json(json_file, lines=True)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = self.data.loc[idx, 'img']
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert('RGB')

        if self.transforms:
            image = self.transforms(image)
        
        # Process image data
        image = feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze()

        # Process text data
        text = self.data.loc[idx, 'text']
        text = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=64)

        label = self.data.loc[idx, 'label']

        return image, text, label


# Define your transforms
transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
])

# Initialize your dataset
dataset = HatefulMemesDataset(json_file="data/train.jsonl",
                              img_dir="data",
                              transforms=transforms)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Validation dataset and DataLoader
val_dataset = HatefulMemesDataset(json_file="data/dev.jsonl",
                                  img_dir="data",
                                  transforms=transforms)

val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Define a simple model for classification
class CombinedModel(nn.Module):
    def __init__(self, text_model, image_model):
        super(CombinedModel, self).__init__()
        self.text_model = text_model
        self.image_model = image_model
        self.classifier = nn.Linear(text_model.config.hidden_size + 768, 2)  # 768 is the output size of vit-base-patch16-224

    def forward(self, image, input_ids, attention_mask):
        text_features = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        image_features = self.image_model(image).last_hidden_state[:, 0, :]
        combined = torch.cat((text_features, image_features), dim=1)
        logits = self.classifier(combined)
        return logits

# Create your model
model = CombinedModel(text_model, image_model)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

# Load the trained model
checkpoint_path = "combined_model_vit_bert_epoch_2.pth"
if torch.cuda.is_available():
    model.load_state_dict(torch.load(checkpoint_path))
else:
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for images, texts, labels in dataloader:
        # Move inputs and targets to the same device as the model
        images = images.permute(1, 0, 2, 3)
        images = images.to(device)
        input_ids = texts['input_ids'].squeeze().to(device)
        attention_mask = texts['attention_mask'].squeeze().to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(images, input_ids, attention_mask)

        # Compute the loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Validation after each epoch
    model.eval()
    with torch.no_grad():
        val_losses = []
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []
        for images, texts, labels in val_dataloader:
            images = images.permute(1, 0, 2, 3)
            images = images.to(device)
            input_ids = texts['input_ids'].squeeze().to(device)
            attention_mask = texts['attention_mask'].squeeze().to(device)
            labels = labels.to(device)

            logits = model(images, input_ids, attention_mask)
            val_loss = criterion(logits, labels)
            val_losses.append(val_loss.item())

            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Store all the labels and predictions for precision, recall, and roc computation
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        avg_val_loss = sum(val_losses) / len(val_losses)
        val_acc = correct / total
        precision = precision_score(all_labels, all_predictions, average='weighted')
        recall = recall_score(all_labels, all_predictions, average='weighted')
        roc_auc = roc_auc_score(all_labels, all_predictions)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Val Loss: {avg_val_loss}, Val Accuracy: {val_acc * 100}%, Precision: {precision * 100}%, Recall: {recall * 100}%, AUROC: {roc_auc}')
    # Save the model after each epoch
    if (epoch + 1) % 5 == 0:
        torch.save(model.state_dict(), f"combined_model_vit_bert_epoch_trial{epoch+1}.pth")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- Th

cuda


KeyboardInterrupt: 

In [18]:
from transformers import AutoTokenizer, AutoModel, ViTModel, ViTFeatureExtractor
import torchvision.models as models
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from PIL import Image
import pandas as pd
import os
import torchvision.transforms as T
from sklearn.metrics import precision_score, recall_score, roc_auc_score


# Transformers model for text
text_model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Torchvision model for images
image_model = ViTModel.from_pretrained("google/vit-base-patch16-224")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

def collate_fn(batch):
    images, texts, labels = zip(*batch)

    # Pad sequences
    input_ids = pad_sequence([t['input_ids'] for t in texts])
    attention_mask = pad_sequence([t['attention_mask'] for t in texts])

    # Convert to tensors
    input_ids = torch.as_tensor(input_ids, dtype=torch.long)
    attention_mask = torch.as_tensor(attention_mask, dtype=torch.long)
    labels = torch.as_tensor(labels, dtype=torch.long)

    # Transpose image tensor (batch_size should be first)
    images = torch.stack(images).transpose(0, 1)
    
    return images, {'input_ids': input_ids, 'attention_mask': attention_mask}, labels


class HatefulMemesDataset(Dataset):
    def __init__(self, json_file, img_dir, transforms=None):
        self.data = pd.read_json(json_file, lines=True)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = self.data.loc[idx, 'img']
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert('RGB')

        if self.transforms:
            image = self.transforms(image)
        
        # Process image data
        image = feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze()

        # Process text data
        text = self.data.loc[idx, 'text']
        text = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=64)

        label = self.data.loc[idx, 'label']

        return image, text, label


# Define your transforms
transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
])

# Initialize your dataset
dataset = HatefulMemesDataset(json_file="data/train.jsonl",
                              img_dir="data",
                              transforms=transforms)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Validation dataset and DataLoader
test_dataset = HatefulMemesDataset(json_file="data/dev_seen.jsonl",
                                  img_dir="data",
                                  transforms=transforms)

test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Define a simple model for classification
class CombinedModel(nn.Module):
    def __init__(self, text_model, image_model):
        super(CombinedModel, self).__init__()
        self.text_model = text_model
        self.image_model = image_model
        self.classifier = nn.Linear(text_model.config.hidden_size + 768, 2)  # 768 is the output size of vit-base-patch16-224

    def forward(self, image, input_ids, attention_mask):
        text_features = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        image_features = self.image_model(image).last_hidden_state[:, 0, :]
        combined = torch.cat((text_features, image_features), dim=1)
        logits = self.classifier(combined)
        return logits

# Create your model
model = CombinedModel(text_model, image_model)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

# Load the trained model
checkpoint_path = "combined_model_vit_bert_epoch_final20.pth"
if torch.cuda.is_available():
    model.load_state_dict(torch.load(checkpoint_path))
else:
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop

# Validation after each epoch
model.eval()
with torch.no_grad():
    test_losses = []
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    for images, texts, labels in test_dataloader:
        images = images.permute(1, 0, 2, 3)
        images = images.to(device)
        input_ids = texts['input_ids'].squeeze().to(device)
        attention_mask = texts['attention_mask'].squeeze().to(device)
        labels = labels.to(device)

        logits = model(images, input_ids, attention_mask)
        test_loss = criterion(logits, labels)
        test_losses.append(test_loss.item())

        _, predicted = torch.max(logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store all the labels and predictions for precision, recall, and roc computation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

    avg_test_loss = sum(test_losses) / len(test_losses)
    test_acc = correct / total
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    roc_auc = roc_auc_score(all_labels, all_predictions)
    print(f'Test Loss: {avg_test_loss}, Test Accuracy: {test_acc * 100}%, Precision: {precision * 100}%, Recall: {recall * 100}%, AUROC: {roc_auc}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- Th

cuda
Test Loss: 0.7439909728746565, Test Accuracy: 50.6%, Precision: 25.6036%, Recall: 50.6%, AUROC: 0.5
